In [1]:
import os
from openai import OpenAI 
from pydantic import BaseModel, Field
import pandas as pd

In [2]:
# OpenAI API 클라이언트 설정
## Set the API key and model name
api_key = "your_api_key"
MODEL = "gpt-4o-2024-08-06"
client = OpenAI(api_key=api_key)

In [3]:
# Pydantic 모델 정의
class Keyword(BaseModel):
    keyword: str
    sentiment: str

In [4]:
class KeywordSentiment(BaseModel):
    keywords: list[Keyword] = Field(..., description="List of extracted keywords with sentiments.")

In [5]:
# CSV 파일 불러오기
def load_csv(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        raise
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        raise

In [10]:
# OpenAI API를 사용하여 리뷰에서 키워드 추출 및 라벨링
def analyze_review_content(content):
    # if client is None:
    #     print("OpenAI client is not initialized.")
    #     return None

    prompt = f"""
    Analyze the following restaurant review and extract 3-5 key phrases representing the review's aspects, such as taste, hygiene, service, and ambiance, in Korean. 
    For each phrase, classify its sentiment as Positive or Negative. Provide the output as a structured JSON format with this schema:
    {{"keywords": [{{"keyword": "<phrase>", "sentiment": "<Positive/Negative>"}}, ...]}}

    Review: {content}
    """


    try:
        completion = client.beta.chat.completions.parse(
            model=MODEL,
            messages=[
                {"role": "system", "content": "You are an assistant that extracts key keywords and their sentiments from restaurant reviews."},
                {"role": "user", "content": prompt}
            ],
            response_format=KeywordSentiment,
        )

        parsed = completion.choices[0].message.parsed
        return parsed.dict()  # Return as a dictionary

    except Exception as e:
        print(f"Error processing content: {content}\n{e}")
        return None

In [11]:
# 리뷰 데이터 전처리 및 키워드 추출 결과 저장
def process_reviews(input_path, output_csv):
    data = load_csv(input_path)

    if 'Review' not in data.columns:
        raise ValueError("The CSV file must contain a 'Review' column.")

    # 리뷰 처리 및 결과 수집
    summaries = []
    for _, row in data.iterrows():
        review = row.get('Review', '')

        # 내용이 비어있으면 건너뜀
        if pd.isna(review) or not review.strip():
            continue

        analysis = analyze_review_content(review)
        if analysis:
            summaries.append({
                "review": review,
                "keywords": analysis.get("keywords", []),
            })

    # 세부 평가 데이터를 행 단위로 변환
    rows = []
    for summary in summaries:
        for keyword_data in summary["keywords"]:
            rows.append({
                "Review": summary["review"],
                "Keyword": keyword_data["keyword"],
                "Sentiment": keyword_data["sentiment"]
            })

    # 결과를 데이터프레임으로 생성
    df_detailed_summary = pd.DataFrame(rows)

    # 결과 미리보기
    print("Generated Results Preview:")
    print(df_detailed_summary.head())

    # 사용자 확인
    save = input("Do you want to save the results? (yes/no): ").strip().lower()
    if save == 'yes':
        try:
            df_detailed_summary.to_csv(output_csv, index=False, encoding="utf-8-sig")
            print(f"Results saved to {output_csv}")
        except Exception as e:
            print(f"Error saving results: {e}")
            raise
    else:
        print("Results not saved.")

In [13]:
# 실행 예시
if __name__ == "__main__":
    base_dir = os.getcwd()  # 현재 작업 디렉토리 경로

    # 입력 파일 및 출력 파일 경로 설정
    input_csv_path = os.path.join(base_dir, "kakaomap review", "이대_reviews_finish.csv")
    output_csv_path = os.path.join(base_dir, "kakaomap review", "이대_with_keywords.csv")

    # 출력 디렉토리 생성
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

    # 리뷰 데이터 처리
    process_reviews(input_csv_path, output_csv_path)

Generated Results Preview:
                                  Review  Keyword Sentiment
0                                최고의 김말이        맛  Positive
1  예전엔 떡볶이가 안 그랬는데 신맛 추가오징어튀김의 오징어가 넘 작음      떡볶이  Negative
2  예전엔 떡볶이가 안 그랬는데 신맛 추가오징어튀김의 오징어가 넘 작음       신맛  Negative
3  예전엔 떡볶이가 안 그랬는데 신맛 추가오징어튀김의 오징어가 넘 작음  추가오징어튀김  Negative
4  예전엔 떡볶이가 안 그랬는데 신맛 추가오징어튀김의 오징어가 넘 작음   오징어 크기  Negative


Do you want to save the results? (yes/no):  yes


Results saved to /Users/yooni/Desktop/TAVE/CamSlien/kakaomap review/이대_with_keywords.csv
